In [2]:
import sys
sys.path.append("..")

from utils.img import transform
from utils.lmdb import LMDBImageDataset, covid_ct_indexer

from torchvision.utils import save_image
import torch
from collections import OrderedDict

usage: ipykernel_launcher.py [-h] [--env ENV]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"eecb8f03-f59a-43dd-a729-eae071288849" --shell=9002 --transport="tcp" --iopub=9004 --f=/tmp/tmp-197331wFJ4CdklyWEF.json


SystemExit: 2

In [2]:
dataset = LMDBImageDataset(config.COVID_19_TRAIN_LMDB, covid_ct_indexer, transform)

In [3]:
lung_raw, lung_targ, med_raw, med_target = dataset[0]

In [5]:
print(torch.stack([lung_raw, lung_targ, med_raw, med_target]).shape)

torch.Size([4, 1, 1024, 1024])


In [15]:
save_image(torch.stack([lung_raw, lung_targ, med_raw, med_target]), "sample.png", normalize=True, value_range=(-1,1))

In [2]:
ckpt = torch.load("../pretrained/stylegan.pt")

In [3]:
ckpt["d"].keys()

odict_keys(['blocks.0.0.weight', 'blocks.0.1.bias', 'blocks.1.conv.0.weight', 'blocks.1.conv.1.bias', 'blocks.1.down_conv.0.kernel', 'blocks.1.down_conv.1.weight', 'blocks.1.down_conv.2.bias', 'blocks.1.skip.0.kernel', 'blocks.1.skip.1.weight', 'blocks.2.conv.0.weight', 'blocks.2.conv.1.bias', 'blocks.2.down_conv.0.kernel', 'blocks.2.down_conv.1.weight', 'blocks.2.down_conv.2.bias', 'blocks.2.skip.0.kernel', 'blocks.2.skip.1.weight', 'blocks.3.conv.0.weight', 'blocks.3.conv.1.bias', 'blocks.3.down_conv.0.kernel', 'blocks.3.down_conv.1.weight', 'blocks.3.down_conv.2.bias', 'blocks.3.skip.0.kernel', 'blocks.3.skip.1.weight', 'blocks.4.conv.0.weight', 'blocks.4.conv.1.bias', 'blocks.4.down_conv.0.kernel', 'blocks.4.down_conv.1.weight', 'blocks.4.down_conv.2.bias', 'blocks.4.skip.0.kernel', 'blocks.4.skip.1.weight', 'blocks.5.conv.0.weight', 'blocks.5.conv.1.bias', 'blocks.5.down_conv.0.kernel', 'blocks.5.down_conv.1.weight', 'blocks.5.down_conv.2.bias', 'blocks.5.skip.0.kernel', 'blocks.5

In [4]:
new_d = OrderedDict()

In [5]:
for k, v in ckpt["d"].items():
    new_k = k
    if k == "final_linear.0.weight":
        new_k = "final_relu.weight"
    if k == "final_linear.0.bias":
        new_k = "final_relu.bias"
    if k == "final_linear.1.weight":
        new_k = "final_linear.weight"
    if k == "final_linear.1.bias":
        new_k = "final_linear.bias"
    new_d[new_k] = v

In [6]:
ckpt["d"] = new_d

In [7]:
torch.save(ckpt, "../pretrained/stylegan_new.pt")